In [52]:
# Libraries
import pandas as pd
import geopandas as gpd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [62]:
# Datasets
od = pd.read_csv("Project_dataset.csv")
samhsa = pd.read_csv("uscities_SAMHSA.csv", encoding="utf-8", encoding_errors="ignore")

In [58]:
od.head()

,State FIPS Code,County FIPS Code,State,FIPS Code,"State, County",County,median_household_income_2023,median_household_income_2022,median_household_income_2021,median_household_income_2020,...,opioid_dispensing_rate_2019,opioid_dispensing_rate_2020,opioid_dispensing_rate_2021,opioid_dispensing_rate_2022,opioid_dispensing_rate_2023,drug_od_count_2019,drug_od_count_2020,drug_od_count_2021,drug_od_count_2022,drug_od_count_2023
0,51,720,VA,51720,"VA, Norton city",Norton city,42618,41298,38316,36004,...,569.1,441.0,390.7,374.9,403.9,0,0,0,0,0
1,51,775,VA,51775,"VA, Salem city",Salem city,63701,63676,60740,70349,...,390.2,377.7,396.4,379.3,342.2,0,0,0,0,0
2,51,690,VA,51690,"VA, Martinsville city",Martinsville city,44547,41500,38571,35715,...,374.8,315.7,302.4,234.3,176.5,0,0,0,0,0
3,21,193,KY,21193,"KY, Perry County",Perry County,42519,33585,31522,33712,...,319.2,284.7,280.5,270.0,264.5,7,18,35,22,25
4,42,93,PA,42093,"PA, Montour County",Montour County,73087,76229,68053,63341,...,298.7,260.2,246.4,241.7,230.0,1,1,5,3,4


In [59]:
od.head()
samhsa.head()

,Program Name,DBA,Street,City,State,zip code,Phone,Certification,FullCertification,County,FIPS_county
0,"Serenity Health, LLC",NaN,780 W. Bel Air Ave Suite B,Aberdeen,MD,21001,(410) 273-1030,Certified,9/30/10,Harford,24025
1,ARS of Aberdeen LLC,ARS of Aberdeen LLC,24-26 West Bel Air Ave.,Aberdeen,MD,21001,(410) 272-0230,Certified,3/18/16,Harford,24025
2,"WCHS, Inc.",Grays Harbor Treatment Solutions,516 East 1st St.,Aberdeen,WA,98520,(877) 774-6565,Certified,5/4/20,Grays Harbor,53027
3,Quinault Indian Nation,Quinault Wellness Center,511 W Heron St,Aberdeen,WA,98520,(564) 544-1950,Certified,8/10/23,Grays Harbor,53027
4,"Abilene Absolute Recovery Solutions, OTP","Abilene Absolute Recovery Solutions, OTP [AARS...",429 North Judge Ely Blvd,Abilene,TX,79601,(325) 400-8586,Certified,4/27/22,Taylor,48059


In [60]:
# Group SAMHSA data by county
samhsa_county = samhsa.groupby("FIPS_county").size().reset_index(name="facility_count")
samhsa_county

,FIPS_county,facility_count
0,1009,1
1,1015,1
2,1019,1
3,1021,1
4,1033,1
...,...,...
763,72025,2
764,72053,1
765,72113,1
766,72127,1


In [ ]:
merged = od.merge(
    samhsa_county,
    left_on="FIPS Code",  # Column name in the 'od' (left) DataFrame
    right_on="FIPS_county",  # Column name in the 'samhsa_county' (right) DataFrame
    how="left",
)
# removing NA's with 0 in case we calculate %
merged["facility_count"] = merged["facility_count"].fillna(0)
merged = merged.rename(columns={"FIPS_county": "samhsa_fips"})

merged = merged.drop(columns="FIPS_county")
merged

# Missing values in smahsa_fips just means there is not a clininc in that fips (county)

,State FIPS Code,County FIPS Code,State,FIPS Code,"State, County",County,median_household_income_2023,median_household_income_2022,median_household_income_2021,median_household_income_2020,...,opioid_dispensing_rate_2021,opioid_dispensing_rate_2022,opioid_dispensing_rate_2023,drug_od_count_2019,drug_od_count_2020,drug_od_count_2021,drug_od_count_2022,drug_od_count_2023,samhsa_fips,facility_count
0,51,720,VA,51720,"VA, Norton city",Norton city,42618,41298,38316,36004,...,390.7,374.9,403.9,0,0,0,0,0,NaN,0.0
1,51,775,VA,51775,"VA, Salem city",Salem city,63701,63676,60740,70349,...,396.4,379.3,342.2,0,0,0,0,0,NaN,0.0
2,51,690,VA,51690,"VA, Martinsville city",Martinsville city,44547,41500,38571,35715,...,302.4,234.3,176.5,0,0,0,0,0,NaN,0.0
3,21,193,KY,21193,"KY, Perry County",Perry County,42519,33585,31522,33712,...,280.5,270.0,264.5,7,18,35,22,25,NaN,0.0
4,42,93,PA,42093,"PA, Montour County",Montour County,73087,76229,68053,63341,...,246.4,241.7,230.0,1,1,5,3,4,42093.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,48,269,TX,48269,"TX, King County",King County,85720,87856,74890,78210,...,NaN,NaN,NaN,0,0,0,0,0,NaN,0.0
3131,48,301,TX,48301,"TX, Loving County",Loving County,86760,94716,100917,97491,...,NaN,NaN,NaN,0,0,0,0,0,NaN,0.0
3132,48,311,TX,48311,"TX, McMullen County",McMullen County,71112,67525,63120,45927,...,NaN,NaN,NaN,0,0,0,0,0,NaN,0.0
3133,2,63,AK,2063,"AK, Chugach Census Area",Chugach Census Area,95180,91356,88029,84063,...,NaN,NaN,NaN,0,0,0,0,0,NaN,0.0
